In [137]:
import xml.etree.ElementTree as ET
import xmltodict

path = '../../../../../bn.qe63.save/'

def read_file(file):
    with open(file, 'r') as myfile:
        xmltext=myfile.read()
    return xmltext

def get_energies_by_band(eigendict, nband):
    banddict = {}
    for i in range(nband):
        banddict[i+1] = []
        for kpoint in range(1,nkpt+1):
            banddict[i+1].append(eigendict[kpoint][i])
    return banddict

def check_if_metallic(banddict):
    metallic = False
    for i in banddict:
        if banddict[i][0] < Fermi:
            bandtype = "valence"
        else:
            bandtype = "conduction"

        for eigenvalue in banddict[i]:
            if eigenvalue < Fermi:
                current_kpt_type = "valence"
            else:
                current_kpt_type = "conduction"
            if bandtype != current_kpt_type:
                metallic = True
    return metallic

#Get dictionary with important info about HOMO/LUMO and metallicity
def get_homo_lumo_dict(eigendict, nkpt, nband, Fermi):
    ev_conv = 27.2114
    homo_bd = []
    homo_eig = []
    lumo_eig = []
    
    for kpt in range(1, nkpt+1):
        for band in range(nband):
            if eigendict[kpt][band] < Fermi:
                highest_val_bd = band +1
                highest_val_en = eigendict[kpt][band]
        homo_bd.append(highest_val_bd)
        homo_eig.append(highest_val_en)
        lumo_eig.append(eigendict[kpt][highest_val_bd])
    
    homo_energy = max(homo_eig)
    homo_kpt = homo_eig.index(max(homo_eig)) + 1
    lumo_energy = min(lumo_eig)
    lumo_kpt = lumo_eig.index(min(lumo_eig)) + 1
    band_gap = lumo_energy - homo_energy
    banddict = get_energies_by_band(eigendict,nband)
    is_metallic = check_if_metallic(banddict)
    #print([x*ev_conv for x in homo_eig],[x*ev_conv for x in lumo_eig])
    
    if homo_kpt == lumo_kpt:
        gap_type = 'DIRECT'
    else:
        gap_type = 'INDIRECT'
    
    
    homo_lumo_dict = {'HOMO_ENERGY': homo_energy*ev_conv, 
                      'LUMO_ENERGY': lumo_energy*ev_conv, 
                      'BAND_GAP': band_gap*ev_conv, 
                      'HOMO_KPT_NUM': homo_kpt, 
                      'LUMO_KPT_NUM': lumo_kpt,
                      'HOMO_BAND_NUM': max(homo_bd),
                      'IS_METALLIC': is_metallic,
                      'FERMI_ENERGY': Fermi*ev_conv,
                      'GAP_TYPE': gap_type}
    return homo_lumo_dict

In [138]:
tree = ET.parse(path + 'data-file-schema.xml')
file = path +  'data-file-schema.xml'
xmldict = xmltodict.parse(read_file(file))

Fermi = float(xmldict['qes:espresso']['output']['band_structure']['fermi_energy'])
nband = int(xmldict['qes:espresso']['output']['band_structure']['nbnd'])
nkpt = int(xmldict['qes:espresso']['output']['band_structure']['nks'])

occdict = {}
eigendict = {}
for i in range(nkpt):
    eigendict[i+1] = [float(q) for q in list(xmldict['qes:espresso']['output']['band_structure']['ks_energies'][i].items())[2][1]['#text'].split(' ')]

In [139]:
get_homo_lumo_dict(eigendict, nkpt, nband, Fermi)

{'HOMO_ENERGY': -0.503765354018746,
 'LUMO_ENERGY': 1.0799229859739077,
 'BAND_GAP': 1.5836883399926536,
 'HOMO_KPT_NUM': 19,
 'LUMO_KPT_NUM': 19,
 'HOMO_BAND_NUM': 13,
 'IS_METALLIC': False,
 'FERMI_ENERGY': 0.09828643441981547,
 'GAP_TYPE': 'DIRECT'}

In [29]:
xmltodict.parse(read_file('../../../../../bn.save/./K00001/eigenval.xml'))['Root']['EIGENVALUES']

OrderedDict([('@type', 'real'),
             ('@size', '17'),
             ('#text',
              '-2.350932570812750E+000\n-1.400700398313944E+000\n-1.393350950316323E+000\n-1.393350765205279E+000\n-6.311616158300198E-001\n-5.834078283397391E-001\n-3.194500575749127E-001\n-2.097309932530080E-001\n-2.097307940084339E-001\n-1.787524662620082E-001\n-1.689689398438817E-001\n-1.689664008106618E-001\n-1.138702308565865E-001\n-1.234747103456562E-002\n-1.234520698282502E-002\n-3.731716212568933E-003\n-3.730487330064607E-003')])

In [72]:
nkpt

19

In [75]:
xmldict = xmltodict.parse(read_file(file))

In [132]:
for i in xmldict['qes:espresso']['output']['band_structure']['ks_energies'][0].items():
    print(i)

('k_point', OrderedDict([('@weight', '1.388888888889e-2'), ('#text', '0.000000000000000e0 0.000000000000000e0 0.000000000000000e0')]))
('npw', '28247')
('eigenvalues', OrderedDict([('@size', '17'), ('#text', '-2.351739143399816e0 -1.363037543108086e0 -1.357002693335830e0 -1.357002693228933e0 -5.701177637689828e-1 -5.295456568793795e-1 -2.300443004931744e-1 -1.033325060920784e-1 -1.033325059620827e-1 -8.977931737616590e-2 -6.405377349834930e-2 -6.405377224980669e-2 -2.779670156534298e-2 9.415064716757684e-2 9.415064820718960e-2 1.024734897687359e-1 1.024734904010048e-1')]))
('occupations', OrderedDict([('@size', '17'), ('#text', '1.000000000000000e0 1.000000000000000e0 1.000000000000000e0 1.000000000000000e0 1.000000000000000e0 1.000000000000000e0 1.000000000000000e0 1.000000000000000e0 1.000000000000000e0 1.000000000000000e0 1.000000000000000e0 1.000000000000000e0 1.000000000000000e0 0.000000000000000e0 0.000000000000000e0 0.000000000000000e0 0.000000000000000e0')]))


In [129]:
[float(q) for q in list(xmldict['qes:espresso']['output']['band_structure']['ks_energies'][0].items())[2][1]['#text'].split(' ')]

-2.351739143399816